In [1]:
import pandas as pd
import numpy as np

# Question 4
# 4_a
Heart = pd.read_csv('Heart.csv', sep=',')
Heart.head()

,age,gender,chest pain type,serum cholestoral,fasting blood sugar,resting electrocardiographic,maximum heart rate achieved,exercise induced angina,target
0,63,1,3,233,1,0,NaN,0,1
1,37,1,2,250,0,1,187.0,0,1
2,41,0,1,204,0,0,172.0,0,1
3,56,1,1,236,0,1,178.0,0,1
4,57,0,0,354,0,1,163.0,1,1


In [2]:
Heart.isna().sum()

age                             0
gender                          0
chest pain type                 0
serum cholestoral               0
fasting blood sugar             0
resting electrocardiographic    0
maximum heart rate achieved     1
exercise induced angina         0
target                          0
dtype: int64

In [3]:
Heart['maximum heart rate achieved'].mean()

149.6456953642384

In [4]:
Heart[Heart['maximum heart rate achieved'].isna()]

,age,gender,chest pain type,serum cholestoral,fasting blood sugar,resting electrocardiographic,maximum heart rate achieved,exercise induced angina,target
0,63,1,3,233,1,0,NaN,0,1


In [5]:
Heart.loc[0, 'maximum heart rate achieved'] = Heart['maximum heart rate achieved'].mean()

In [6]:
Heart.isna().sum()

age                             0
gender                          0
chest pain type                 0
serum cholestoral               0
fasting blood sugar             0
resting electrocardiographic    0
maximum heart rate achieved     0
exercise induced angina         0
target                          0
dtype: int64

In [7]:
import jenkspy
# 4_b
breaks = jenkspy.jenks_breaks(Heart['age'], nb_class=3)
print(breaks)

l = ['29_49','49_60','60_77']
Heart['age'] = pd.cut(Heart['age'], bins=breaks, labels=l)

[29.0, 49.0, 60.0, 77.0]


In [8]:

breaks = jenkspy.jenks_breaks(Heart['maximum heart rate achieved'], nb_class=3)
print(breaks)

l = ['71_133','133_160','160_202']
Heart['maximum heart rate achieved'] = pd.cut(Heart['maximum heart rate achieved'], bins=breaks, labels=l)

[71.0, 133.0, 160.0, 202.0]


In [9]:

breaks = jenkspy.jenks_breaks(Heart['serum cholestoral '], nb_class=3)
print(breaks)

l = ['126_226','226_290','290_564']
Heart['serum cholestoral '] = pd.cut(Heart['serum cholestoral '], bins=breaks, labels=l)

[126.0, 226.0, 290.0, 564.0]


In [10]:
Heart =  pd.get_dummies(Heart,columns=Heart.columns)

In [11]:
Heart.head()

,age_29_49,age_49_60,age_60_77,gender_0,gender_1,chest pain type_0,chest pain type_1,chest pain type_2,chest pain type_3,serum cholestoral _126_226,...,resting electrocardiographic_0,resting electrocardiographic_1,resting electrocardiographic_2,maximum heart rate achieved_71_133,maximum heart rate achieved_133_160,maximum heart rate achieved_160_202,exercise induced angina_0,exercise induced angina_1,target_0,target_1
0,0,0,1,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,1
1,1,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,1,0,0,1
2,1,0,0,1,0,0,1,0,0,1,...,1,0,0,0,0,1,1,0,0,1
3,0,1,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,1,0,0,1
4,0,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1


In [12]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

frq_items_apriori = apriori(Heart, min_support = 0.2, use_colnames = True)
frq_items_apriori.sort_values(by=['support'], inplace=True)
frq_items_apriori

,support,itemsets
22,0.201320,"(age_29_49, target_1)"
116,0.201320,"(target_0, resting electrocardiographic_0, che..."
126,0.201320,"(maximum heart rate achieved_160_202, fasting ..."
119,0.204620,"(chest pain type_2, exercise induced angina_0,..."
112,0.204620,"(exercise induced angina_1, gender_1, target_0)"
...,...,...
39,0.574257,"(gender_1, fasting blood sugar_0)"
72,0.577558,"(exercise induced angina_0, fasting blood suga..."
15,0.673267,(exercise induced angina_0)
4,0.683168,(gender_1)


In [13]:
# 4_c
rules = association_rules(frq_items_apriori, metric ="lift", min_threshold = 1)
no_stroke = rules[rules['consequents'] == frozenset({'target_0'})]
stroke    = rules[rules['consequents'] == frozenset({'target_1'})]


In [14]:
display(no_stroke.head())
display(stroke.head())

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(resting electrocardiographic_0, chest pain ty...",(target_0),0.257426,0.455446,0.201320,0.782051,1.717113,0.084077,2.498544
20,"(exercise induced angina_1, gender_1)",(target_0),0.254125,0.455446,0.204620,0.805195,1.767928,0.088880,2.795380
36,"(exercise induced angina_1, fasting blood suga...",(target_0),0.273927,0.455446,0.204620,0.746988,1.640126,0.079861,2.152287
79,"(resting electrocardiographic_0, fasting blood...",(target_0),0.399340,0.455446,0.211221,0.528926,1.161337,0.029344,1.155984
104,"(gender_1, resting electrocardiographic_0)",(target_0),0.339934,0.455446,0.217822,0.640777,1.406923,0.063000,1.515922


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(age_29_49),(target_1),0.287129,0.544554,0.201320,0.701149,1.287565,0.044963,1.523991
14,"(exercise induced angina_0, chest pain type_2)",(target_1),0.250825,0.544554,0.204620,0.815789,1.498086,0.068033,2.472419
40,"(exercise induced angina_0, gender_1, fasting ...",(target_1),0.356436,0.544554,0.204620,0.574074,1.054209,0.010522,1.069307
48,"(serum cholestoral _126_226, fasting blood sug...",(target_1),0.323432,0.544554,0.204620,0.632653,1.161781,0.028494,1.239824
72,"(exercise induced angina_0, gender_0)",(target_1),0.244224,0.544554,0.211221,0.864865,1.588206,0.078228,3.370297
